# Homework 5 - Visit the Wikipedia hyperlinks graph!

At the first step we need to import labrary and open the file

In [1]:
from collections import defaultdict
import math
import statistics
from collections import deque
import pickle
import os
#os.chdir("C:\\Users\\user\\Documents\\HW5_ADM")

**[RQ 1]**

** HYPERLINKS**

We import the links and we store those pairs insiede a list first, and then we crated two dictionaries. First dictionary contains sources as a keys and destinations as values, instead the second one is the opposite.

In [2]:
with open("wiki-topcats-reduced.txt", 'r') as fl:
    lista_1= []#list of link pairs 
    Nodes=[]#list of nodes
    for line in fl:
        i = [s.strip() for s in line.split("\t")]
        lista_1.append(i)
        Nodes.extend(i)

In [3]:
Nodes=set(Nodes)#we convert the list of nodes to set, because of the next steps

In [4]:
dict_hl1 = defaultdict(list)#source to destination
for k,v in lista_1:
    dict_hl1[str(k)].append(str(v))

In [5]:
dict_hl2 = defaultdict(list)#destination to source
for k,v in lista_1:
    dict_hl2[v].append(k)

In [6]:
print('The number of sources is: ' + str(len(dict_hl1)) + ' and number of destinations is: ' + str(len(dict_hl2)),
      '\nThe number of nodes of our graph is: ' + str(len(Nodes)),
      '\nThe number of edges is: ' + str(len(lista_1)),
      '\nThe density is: ' + str(len(lista_1)/(len(Nodes)*(len(Nodes)-1))),
      '\nThe average degree is: ' + str(len(lista_1)/len(Nodes)))

The number of sources is: 428957 and number of destinations is: 352518 
The number of nodes of our graph is: 461193 
The number of edges is: 2645247 
The density is: 1.2436602635647606e-05 
The average degree is: 5.735661642739591


Since number of sources and destinations are not the same we can conclude that it's a directed graph.

**CATEGORIES**

Here we opened the file taht contains categories' lists and we store it in a dicionary where the keys are the categories and values are list of articles. 

In [7]:
with open('wiki-topcats-categories.txt', 'r') as fil:
    lista_3 = []#it's a list of lists where the first value is tha name of categories and the others are the index of the articles
    for line in fil:
        i = [s.strip() for s in line.split()]
        lista_3.append(i)            

In [8]:
for lst in lista_3:
    lst[0] = lst[0][9:-1]#we just clean the name of categories just to be more easy to read.

In [9]:
dict_3 = {}#get a dictionary where the key is the category and value is the list of articols
for v in lista_3:
    dict_3[v[0]]=set(v[1:]).intersection(Nodes)#we take the intersection because we need only nodes that were mentioned in the reduced graph which we are working.

In [10]:
dict_4 = {}#now we have a dictionary with only the categories with at least 3500 articols
for k in dict_3:
    #print(dict_3.)
    if len(dict_3.get(k)) >= 3500:
        dict_4[k] = dict_3.get(k)
    else:
        continue
print('The number of categories to compare is: ' + str(len(dict_4)))# finally we have only 29 categories to compare.

The number of categories to compare is: 29


**SHORTEST PATH FUNCTION**

Now we defined our function to calculate the shortest path between two nodes. <br/>
The function takes 2 values, the starting node and the ending node. 

In [11]:
def bfs_shortest_path(start, goal):

    visited = []#nodes checked before reaching the goal
    queue = deque([start])#list of nodes to be checked
    distance = 0#that's our counter, to count the number of steps to reach the goal node
        
    while queue:
        #we opened a loop, that it will be stopped only when the len of the queue will be 0.
        #It will appen only when all the nodes connected to our starting point will be checked.
        path = queue.popleft()#delete the first item from the queue that must be analysed.

        if path not in visited:
            #if the node is not in visited list, start to analyse it.
            neighbours = dict_hl1[path]
            #we take all the nodes directly connected to the node that we are analysing
            
                        
            for neighbour in neighbours:
                queue.append(neighbour)
                #append all the neightbours to the queue because we need to check those in the future if 
                #we'll not find our goal in this time
                    
                distance +=1#add one to our counter
                if distance >500:
                # we put a condition, just to make our analyses faster, so if the distance is grater than 500 stop.
                #We decided to do that because we 're looking for the shortest so we don't need to store 
                #all the possibile paths.
                    return('path > 500')

                elif neighbour == goal:
                    #if the the neighbour == goal stop because you've fuound the goal node.
                    return distance
                
            visited.append(path)
            #store the analysed node inside the list because we don't wanna analyse it again.

    else:
        return ('No path found')
        #if the queue si empty it means that there are no paths to reach the goal.

Here we implemented function to calculate all possible shortest paths between two categories.

In [12]:
def calc_sh_p(c0,ci):
    shortest_path_2 = defaultdict(list)#we defined a dictionary for storing values.The keys are nodes in input categories and values are paths to nodes to i-th category.
    
    for n in c0:# per each node of input category we started to calculate all the shortest paths to i-th category
        for n_1 in ci:#we calculate the paths per each node of i-th category

            if n == n_1:
                shortest_path_2[n].append(0)
                #if the source and destination are the same just add 0 and continue with the next node in i-th category.
                break
            elif len(shortest_path_2[n]) > 9:
    #we decided to add another condition to make the compute faster.
    #we decided to take only 9 possibile paths to node in i-th category, because we'll take only one of those,
    #more precisely the shortest.
                    break
            else:
                value = bfs_shortest_path(n, n_1)
                # here we store the path inside a variable that it will be appended to the dictionary. 
                if value == 'No path found' or value == 'path > 500':
                    #if the function gives a path greater than 500 or there is no path we store nothing
                    continue

                else:
                    shortest_path_2[n].append(value)#if the function gives us a right path we'll store it into the dictionary.
    return shortest_path_2

Here we chose input category and than create a list with names of other categories. Then we executed functions for storing all possible pathes between input and other categories.
During each iteration we stored every dictionary into a .pkl file, in this way we can run the function just one time.

In [13]:
C0=dict_4['Association_football_goalkeepers']#input category

slist=[i for i in dict_4.keys() if i != 'Association_football_goalkeepers']#list of all categories except input category

In [14]:
#that's our function to store every dictionary into a .pkl file
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
#we store all the dictionaries categories' pairs into single files.
#(if you don't wanna run it, you can find in the repository a zip file with all of them).
for x in slist:
    sh=calc_sh_p(C0,dict_4[x])#calculation of possible pathes
    save_obj(sh,str(x))#storing to file


In [15]:
#that's our function to load .pkl files
def load_obj(name ):
    with open('Dictionaries of category pairs/'+ name + '.pkl', 'rb') as f:
        return pickle.load(f)

**[RQ 2]**

**Block-Ranking**

In [16]:
#this is the function to get the median from values of one dictionary.
def get_median1(d_sh):
    list_of_pathes=[]
    for x in d_sh.keys():
        if d_sh[x]==[]:
            list_of_pathes.append(461194)#if there is no path we put this number because it's impossible to go through more than total number of nodes.
        else:
            list_of_pathes.extend(d_sh[x])#we extend our list if there are same pathes.
    return statistics.median(list_of_pathes)#here we take the median of that list.

In [17]:
#here we calculate our ranking vector as dictionary where the keys are categories' names and values will be medians.
all_rank_vec={}
for x in slist:
    namei=load_obj(x)#loading file that contains passible shortest pathes
    ranki=get_median1(namei)#getting the median
    all_rank_vec[x]=ranki#storing into dictionary.

In [18]:
#to sort our dictionary by values. 
Rank_list=sorted(all_rank_vec.items(), key=lambda kv: kv[1])
Rank_list

[('Association_football_forwards', 157),
 ('Association_football_midfielders', 234.0),
 ('Living_people', 242.0),
 ('Place_of_birth_missing_(living_people)', 263),
 ('Year_of_birth_missing', 283),
 ('Year_of_birth_missing_(living_people)', 284.0),
 ('Year_of_death_missing', 287.0),
 ('English_footballers', 344),
 ('The_Football_League_players', 354.0),
 ('Association_football_defenders', 376.0),
 ('Harvard_University_alumni', 461194.0),
 ('Major_League_Baseball_pitchers', 461194.0),
 ('Members_of_the_United_Kingdom_Parliament_for_English_constituencies',
  461194),
 ('Indian_films', 461194.0),
 ('Rivers_of_Romania', 461194),
 ('Main_Belt_asteroids', 461194),
 ('Asteroids_named_for_people', 461194),
 ('English-language_albums', 461194),
 ('British_films', 461194),
 ('English-language_films', 461194.0),
 ('American_films', 461194.0),
 ('People_from_New_York_City', 461194.0),
 ('American_television_actors', 461194.0),
 ('American_film_actors', 461194.0),
 ('Debut_albums', 461194),
 ('Blac

As the input category we chose 'Association_football_goalkeepers', and from the output above it is clear that the closest categories are topics related to football and the farthest are topics related to other sports, films, science and history.

In [19]:
#getting sorted names in order to do following calculations.
ranked_names=[]
for i in Rank_list:
    ranked_names.append(i[0])
ranked_names

['Association_football_forwards',
 'Association_football_midfielders',
 'Living_people',
 'Place_of_birth_missing_(living_people)',
 'Year_of_birth_missing',
 'Year_of_birth_missing_(living_people)',
 'Year_of_death_missing',
 'English_footballers',
 'The_Football_League_players',
 'Association_football_defenders',
 'Harvard_University_alumni',
 'Major_League_Baseball_pitchers',
 'Members_of_the_United_Kingdom_Parliament_for_English_constituencies',
 'Indian_films',
 'Rivers_of_Romania',
 'Main_Belt_asteroids',
 'Asteroids_named_for_people',
 'English-language_albums',
 'British_films',
 'English-language_films',
 'American_films',
 'People_from_New_York_City',
 'American_television_actors',
 'American_film_actors',
 'Debut_albums',
 'Black-and-white_films',
 'American_military_personnel_of_World_War_II',
 'Windows_games']

**COMPUTING SUBGRAPHS**

In [20]:
def calc_sub(C00):# computing subgraph from input category.
    C0_r={}
    # we consider that at the beginnig all edges has 1 as weight
    for i in C00:
        if i in dict_hl2.keys():
            rr=len(set(C00).intersection(set(dict_hl2[i])))#we take number of in-edges
            if rr>0:
                C0_r[i]=rr
    return C0_r



def transform_sub(C0_r,C00):#function for preprocessing to next calculation
    WC0={}
    t=[]
    #we take each node in subgraph
    for x in C0_r.keys():
        if x in dict_hl1.keys():# if it sends edges
            for i in set(dict_hl1[x]).intersection(C00):#for each node that gets edge we store the score of sending edge as weight
                if i in WC0:# if node has several in-edges we store several weights
                    t=WC0[i]
                    t.append(C0_r[x])
                    WC0[i]=t
                else:
                    WC0[i]=[C0_r[x]]
    return WC0



def sub_calc_2(C00,C1,WC0):# function to calculate score of nodes in subgraph with adding nodes from next category
    C0_0=C00.union(C1)#adding nodes of the next category
    C1_r={}
    a=0#the default value for sum of in-edges from previous subgraph 
    for i in C0_0:
        if i in dict_hl2.keys():#if node has in-edges
            if i in WC0:#if node was in previous subgraph and had in-edges
                a=sum(WC0[i])-len(WC0[i])# we take sum of weights ans we substract their count in order to avoid repetiton
            val=a+len(set(dict_hl2[i]).intersection(C1))#taking the sum of weights
            if val>0:
                C1_r[i]=val#we store only values that are more than 0
    return C1_r


In [21]:
C0=dict_4['Association_football_goalkeepers']# input category

**[STEP 1]**

In [22]:
sub0=calc_sub(C0) # calculating scores for first subgraph

In [23]:
len(C0) # number of nodes in fisrt subgraph

3737

In [24]:
save_obj(sub0,"firstSubgraph") # saving into file

In [25]:
sub0tramsf=transform_sub(sub0,C0) # getting weights of edges for the next computing bigger subgraph

**[STEP 2]**

In [26]:
C_1=ranked_names[0] #name of the next category

In [27]:
C_1

'Association_football_forwards'

In [28]:
sub0_1=sub_calc_2(C0,set(dict_4[C_1]),sub0tramsf) #computing scores for subgraph that contains nodes 
#from the first graph and the first category of our block ranking

In [29]:
save_obj(sub0_1,"secondSubgraph")

In [30]:
C0=C0.union(dict_4[C_1])# adding nodes from first category of block ranking to input category

In [31]:
len(C0) # number of nodes in the secod subgraph

8827

In [32]:
len(sub0_1) # the number of nodes with in-edges

5198

**[STEP 3]**

In [33]:
sub0tramsf1=transform_sub(sub0_1,C0)# preproccessing of the second graph for the next iteration

In [34]:
g0=sub0_1 #second subgraph
gc0=sub0tramsf1# weights of edges from second subgraph
c=2#that's the counter
C0=set(C0)
for x in ranked_names[1:]: # here should be our ranking vector
    c=c+1
    C11=set(dict_4[x])
    sub=sub_calc_2(C0,C11,gc0)#getting subgraph
    save_obj(sub,str(c)+"_sub_"+x)#saving into file
    C0=C0.union(C11)#add nodes to set of nodes of the prevous graph
    gc0=transform_sub(sub,C0)# getting weights of edges to compute scores for next graph
    print("adding the nodes from a category: "+ str(c)+" number of nodes: "+ str(len(C0))+
         "\n number of nodes with in edges, that means they have a score: "+str(len(sub))) # the number of nodes in subgraph

adding the nodes from a category: 3 number of nodes: 14484
 number of nodes with in edges, that means they have a score: 9000
adding the nodes from a category: 4 number of nodes: 349696
 number of nodes with in edges, that means they have a score: 262209
adding the nodes from a category: 5 number of nodes: 349758
 number of nodes with in edges, that means they have a score: 262062
adding the nodes from a category: 6 number of nodes: 353867
 number of nodes with in edges, that means they have a score: 264855
adding the nodes from a category: 7 number of nodes: 353928
 number of nodes with in edges, that means they have a score: 265091
adding the nodes from a category: 8 number of nodes: 356276
 number of nodes with in edges, that means they have a score: 266358
adding the nodes from a category: 9 number of nodes: 357580
 number of nodes with in edges, that means they have a score: 267360
adding the nodes from a category: 10 number of nodes: 357880
 number of nodes with in edges, that me

Now we print 30 nodes of the last subgraph, where there is the score per each node

In [38]:
stop = 0
for k,v in sub.items():
    if stop == 30:
        break
    print(str(k) +' The score of this node is: ' + str(v))
    stop += 1


45819 The score of this node is: 31448106212962684043734196557061984449787
1591276 The score of this node is: 375276843772144871886988461680898508276951
1371112 The score of this node is: 375276843772144871886988461680898508276951
724983 The score of this node is: 10631491339783285184273653253056319935295
1196929 The score of this node is: 285974148439555534540491711566358016720336
786730 The score of this node is: 17566296542326346503468122607592194095
1602913 The score of this node is: 266872439095612402421049474129436882923315
581239 The score of this node is: 53094465197549764440971449781549594879851
668691 The score of this node is: 96313464409665255037263672270165045381202201
1469821 The score of this node is: 5601327600969060894322021706152323426924862
996339 The score of this node is: 17678423591677957728202351437362689737
873959 The score of this node is: 362246557833235628918521160732855
1273455 The score of this node is: 72745195579684980031610708435840854521
1006941 The sco